<img src="./ESSENCE.png" width=700px>

### Reflection pattern

## input -> function(input) -> output -> function(output) -> output2

<img src="./INPUT_OUTPUT.png" width=700px>




## Agent One is a code expert. Agent Two is an expert code reviewer.

We generate a response with our first query using a system prompt to create code.

We then pass the output into another function that acts as a reviewer to produce the next version of the code.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:14]}...")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-TVyeca...


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)
# Output below...

Certainly! Below is a simple yet comprehensive implementation of making a GET request to an API using the `requests` library in Python. This example demonstrates how to make the request, handle potential errors, and print out the response.

```python
import requests

def fetch_data_from_api(url):
    """
    Fetch data from the specified API URL.

    :param url: The API endpoint URL to request data from.
    :return: JSON response if the request was successful, None otherwise.
    """
    try:
        # Sending a GET request to the API
        response = requests.get(url)

        # Raise an exception for HTTP errors
        response.raise_for_status()

        # Return the JSON response
        return response.json()
        
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")
    return None

if __name__ == "__main__":
    # Example usage
    api_url = "https://api.example.com/data"  # Replace with a real API URL
    data = fetch_data_from_api(api_url)
    
    if data is not None:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data.")
```

### Explanation:
1. **Importing Requests**: We import the `requests` library to facilitate making HTTP requests.
2. **Function Definition**: The function `fetch_data_from_api` is defined to make it reusable.
3. **Error Handling**: Various exceptions are caught to handle different types of errors that might occur during the request.
4. **Main Block**: The `if __name__ == "__main__":` block allows us to run the example code when the script is executed directly.

Make sure you have the `requests` library installed in your environment; you can install it via `pip`:

```bash
pip install requests
```

Feel free to run this code in an appropriate Python environment, and replace the `api_url` with a valid endpoint for testing.

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT and we change the system prompt or what we want it to do along with the output from the previous query.


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)
# Critique displayed below...

Your code for making a GET request to an API using the `requests` library is well structured and easy to follow. Here are some critiques and recommendations to further improve it:

### Strengths:
1. **Clear Functionality**: The function has a well-defined purpose: to fetch data from an API.
   
2. **Error Handling**: You've incorporated thorough error handling for different exception types, which is important for debugging and maintaining robustness in network operations.

3. **Usage of `raise_for_status()`**: This method automatically raises an HTTPError for bad responses (4xx and 5xx), which simplifies error handling.

4. **Documentation**: The docstring effectively communicates the function’s parameters and return type.

### Recommendations for Improvements:

1. **Logging Instead of Printing**:
   - Instead of using `print()` statements for error handling, consider using the `logging` module. This allows for better control of how error messages are recorded and can be adjusted for different environments (development vs. production).
   ```python
   import logging
   logging.basicConfig(level=logging.ERROR)
   ...
   logging.error(f"HTTP error occurred: {http_err}")
   ```

2. **Timeouts**:
   - It's good practice to set a timeout when making API calls. This prevents the request from hanging indefinitely. The `get()` method can take a `timeout` parameter.
   ```python
   response = requests.get(url, timeout=10)  # 10 seconds timeout
   ```

3. **Return Type Clarity**:
   - Although you return `None` for failed requests, it would be better to keep the return type consistent. Consider returning an empty dictionary instead of `None`, unless `None` signifies something specific in your application.
   ```python
   return {}
   ```

4. **Type Hinting**:
   - Adding type hints can increase the readability of your code and improve the usability of the function in larger projects.
   ```python
   def fetch_data_from_api(url: str) -> dict:
   ```

5. **Configurable API URL**:
   - Instead of hardcoding the API URL, consider making it configurable (e.g., via environment variables or a configuration file). This enhances the flexibility of your code when deploying to different environments.

6. **Add Tests**:
   - If this is part of a larger application, consider writing unit tests. You can use `unittest` or `pytest` to test this function with both valid and invalid URLs, checking that it handles each case as expected.

7. **Consider JSON Decoding Exceptions**:
   - Handle cases where the response might not be valid JSON. You can catch `ValueError` when trying to decode the response.
   ```python
   return response.json()  # This can raise ValueError if the response is not valid JSON
   ```

### Revised Example Code:

Here’s a slightly modified version of your code incorporating some of the above recommendations:

```python
import requests
import logging

logging.basicConfig(level=logging.ERROR)

def fetch_data_from_api(url: str) -> dict:
    """
    Fetch data from the specified API URL.

    :param url: The API endpoint URL to request data from.
    :return: JSON response as dictionary if the request was successful, empty dict otherwise.
    """
    try:
        response = requests.get(url, timeout=10)  # Set a timeout
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logging.error(f"An error occurred: {req_err}")
    except ValueError as json_err:
        logging.error(f"JSON decoding error: {json_err}")

    return {}  # Return an empty dict instead of None

if __name__ == "__main__":
    api_url = "https://api.example.com/data"  # Replace with a real API URL
    data = fetch_data_from_api(api_url)
    
    if data:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data.")
```

Your implementation is off to a great start! With these adjustments, you'll enhance its robustness and usability.

Add CRITIQUE to chat...

Notice how we are appending previous responses so that the next SYSTEM MESSAGE has history or context.


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)
# Response to critique displayed below...

Thank you for your valuable feedback! Here’s the revised version of the previous implementation for making a GET request to an API using the `requests` library, incorporating your suggestions for improvement.

### Revised Python Implementation

```python
import requests
import logging

# Set up logging to capture errors
logging.basicConfig(level=logging.ERROR)

def fetch_data_from_api(url: str) -> dict:
    """
    Fetch data from the specified API URL.

    :param url: The API endpoint URL to request data from.
    :return: JSON response as a dictionary if the request was successful, empty dict otherwise.
    """
    try:
        # Sending a GET request with a timeout
        response = requests.get(url, timeout=10)  # Set a timeout

        # Raise an exception for HTTP errors
        response.raise_for_status()

        # Attempt to return the JSON response
        return response.json()
        
    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logging.error(f"An unexpected error occurred: {req_err}")
    except ValueError as json_err:
        logging.error(f"JSON decoding error: {json_err}")

    return {}  # Return an empty dict instead of None

if __name__ == "__main__":
    # Example usage
    api_url = "https://api.example.com/data"  # Replace with a real API URL
    data = fetch_data_from_api(api_url)
    
    if data:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data.")
```

### Improvements Included:
1. **Logging**: Integrated the `logging` module for improved error reporting instead of using prints.
2. **Timeouts**: Implemented a timeout for the request to prevent indefinite waiting.
3. **Return Type**: Changed the return value on failure to an empty dictionary to provide consistency in return type.
4. **Type Hinting**: Added type hints to improve clarity regarding function parameters and return types.
5. **Error Handling for JSON Decoding**: Added a `ValueError` exception handling to capture issues arising from invalid JSON responses.

These changes enhance the code's usability and robustness, especially in production environments. Thank you for your insightful critique!

<br>

### We can of course make this a Class...
